In [30]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

init_notebook_mode(connected=True)
plotly.__version__

'3.8.1'

In [47]:
tweets = pd.read_pickle('../datasets/TweetsDataset.pkl')
users = pd.read_pickle('../datasets/UsersDataset.pkl')
print(tweets.columns)
display(users.head(3))
tweets.head()

Index(['tweet_id', 'user_id', 'screen_name', 'complete_name', 'text',
       'hashtags', 'mencioned_user_id', 'mencioned_screen_name', 'is_reply',
       'reply_screen_name', 'reply_user_id', 'all_reply_screen_name',
       'all_reply_user_id', 'is_retweet', 'retweet_user_id',
       'retweet_screen_name', 'retweet_complete_name', 'retweet_count',
       'reply_count', 'favorite_count', 'num_hashtags', 'num_urls',
       'num_mentions', 'created_at', 'crawled_at'],
      dtype='object')


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
index,,,,,,,,,,,,,,,
2300560809,2300560809,NMoraniB,Nicole Morani Brown,593,Gerencia General de @PuertoBsAs Ministerio de ...,"Buenos Aires, Argentina",False,False,0,2014-01-01 00:00:00,2019-06-29 23:04:21.059135,2377,1630,1510,False
1536923558,1536923558,FabianPereyra87,Fabian Pereyra,2532,,False,False,False,0,2013-06-01 00:00:00,2019-06-29 23:04:21.289518,4307,53,284,False
235491971,235491971,gsrcaballero,Gus Caballero,21635,"Papa de Cami y Rochi, FANA de La Academia, PRO...",False,False,False,0,2011-01-01 00:00:00,2019-06-29 23:04:21.030207,5683,1691,1360,False


,tweet_id,user_id,screen_name,complete_name,text,hashtags,mencioned_user_id,mencioned_screen_name,is_reply,reply_screen_name,...,retweet_screen_name,retweet_complete_name,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,created_at,crawled_at
1146580422798860288,1146580422798860288,291506013,lunaenpluton,Ocasi 💚,"“Fito Paaaaaeeeeezzzz, donde estaaaaaaaas?” Es...",[],[138072143],[FitoPaezMusica],False,NaN,...,NaN,NaN,0,1,17,0,0,1,2019-07-04 00:44:09,2019-07-08 11:53:25.184210
1145744849636876294,1145744849636876294,181239585,cocainfame,💚Coca💚,-Le di de comer y dormí al bebe\n-Hice tarta g...,[],[],[],False,NaN,...,NaN,NaN,445,772,3468,0,0,0,2019-07-01 17:23:53,2019-07-08 13:05:38.377025
1148208276959584257,1148208276959584257,2312698567,cadenainforma,CADENA INFORMATIVA,Macri encabeza en Parque Norte el lanzamiento ...,[],[],[],False,NaN,...,NaN,NaN,5,1,9,0,2,0,2019-07-08 12:32:40,2019-07-08 11:53:27.285528
1145145462518898688,1145145462518898688,291506013,lunaenpluton,Ocasi 💚,“Quiero escuchar Fito Páez”. Tiene un año y oc...,[],[],[],False,NaN,...,NaN,NaN,0,1,13,0,0,0,2019-06-30 01:42:08,2019-07-08 11:53:41.788105
891083958712250368,891083958712250368,351400701,lzlptclss,luis islas,Cara de tener la conciencia rota,[],[],[],False,NaN,...,NaN,NaN,0,0,1,0,0,0,2017-07-28 23:52:25,2019-07-08 11:53:29.922502


In [114]:
#tweets_humans = tweets[tweets.Cat=='Humans']
#display(tweets_humans.columns)

#first = True
def getTimes(times):
    if(len(times) == 0):
        return np.array([0,0,0,0])
    return np.array([times.mean().seconds, times.median().seconds, times.std().seconds, times.quantile(0.25).seconds, times.quantile(0.75).seconds])

def getDist(var):
    if(len(var) == 0):
        return np.array([0,0,0,0])
    return np.array([var.mean(), var.median(), var.std(), var.quantile(0.25), var.quantile(0.75)])

def getWeekAv(tw):
    return len(tw) / len(tw.weekNum.unique())

def getDataTweets(twU):
    userId = twU.user_id.values[0]
    twU = twU.sort_values(by='created_at', ascending=0).fillna(0) #tweets del usuario
    dates = twU.created_at #aca lo que hago es guardar la columnoa como datetime

    twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
    twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
    twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))

    tweetsUser = twU[twU.is_retweet==False] #tweets propios del usuario
    tweetsRT = twU[twU.is_retweet==True] #retweets

#     twU.loc[:,'interval'] = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp)
    twU.loc[:,'interval'] = pd.to_datetime(twU.created_at.shift(1)) - pd.to_datetime(twU.created_at)
    times = pd.to_datetime(twU.created_at.shift(1)) - pd.to_datetime(twU.created_at) # calculo distancia entre tweets

    tweetsUser.loc[:,'interval'] = pd.to_datetime(tweetsUser.created_at.shift(1)) - pd.to_datetime(tweetsUser.created_at)
    timesUser = pd.to_datetime(tweetsUser.created_at.shift(1)) - pd.to_datetime(tweetsUser.created_at) # calculo distancia entre tweets

    tweetsRT.loc[:,'interval'] = pd.to_datetime(tweetsRT.created_at.shift(1)) - pd.to_datetime(tweetsRT.created_at)
    timesRT = pd.to_datetime(tweetsRT.created_at.shift(1)) - pd.to_datetime(tweetsRT.created_at) # calculo distancia entre tweets
    
    mediaSemanas = np.around(twU.groupby('dayOfWeek').apply(getWeekAv), 6)
    
    #ver word_to_vect
    result = np.concatenate([
        [
            userId,
            twU.tweet_id.count(),
            len(twU.weekNum.unique())
        ],
        mediaSemanas,
        getTimes(times),
        getTimes(timesRT),
        getTimes(timesUser),
        getDist(tweetsUser.favorite_count.astype(int)) * 10000,
        getDist(tweetsUser.retweet_count.astype(int)) * 10000,
        getDist(tweetsUser.num_mentions.astype(int)) * 10000,
        getDist(tweetsUser.reply_count.astype(int)) * 10000,

    ])
    
    return result

res = tweets.groupby('user_id').apply(getDataTweets)
cols = np.concatenate(
    [
        
        [x+'_mean',x+'_median', x+'_std', x+'_fq', x+'_tq'] for x in
        ['times','timesRT','timesUser','favorite_count','retweet_count','num_mentions','reply_count']
    ]
)
cols = np.concatenate([
    ['user_id', 'tweetsCount', 'semanas', 'lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom'],cols
])
res = pd.DataFrame(
    [x for x in res], columns = cols
)
res.index = res.user_id

print(len(res))

1486


In [124]:
res.head()

,user_id,tweetsCount,semanas,lun,mar,mie,jue,vie,sab,dom,...,num_mentions_mean,num_mentions_median,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq
user_id,,,,,,,,,,,,,,,,,,,,,
102407528,102407528,170,10,3.166667,3.666667,3.833333,5.142857,4.625,3.666667,2.75,...,833.3333333333333,0.0,2776.739162008521,0.0,0.0,21574.074074074073,10000.0,31330.67640797719,0.0,30000.0
1032436344759504896,1032436344759504896,87,4,3.666667,3.5,3.0,4.5,6.0,5.666667,10.333333,...,0,0,0,0,0,None,None,None,None,None
1032441989915140096,1032441989915140096,178,18,2.333333,2.166667,2.0,2.555556,1.666667,2.416667,1.636364,...,333.3333333333333,0.0,2141.5250999964933,0.0,0.0,2000.0,0.0,4634.275570907933,0.0,0.0
1032737865602674690,1032737865602674690,177,9,2.75,7.333333,2.0,7.333333,14.333333,11.75,6.5,...,0.0,0.0,0.0,0.0,0.0,1904.7619047619046,0.0,4822.86981579155,0.0,0.0
1032811143553789952,1032811143553789952,111,3,6.5,24.0,24.0,16.0,4.0,4.0,9.0,...,0.0,0.0,nan,0.0,0.0,0.0,0.0,nan,0.0,0.0


In [1]:
import pickle
with open('../modelo/modelTweets.pkl', 'rb') as m_pred:
    model = pickle.load(m_pred)
model

Pipeline(memory=None,
     steps=[('cat', <catboost.core.CatBoostClassifier object at 0x00000272FD668860>)])

In [231]:
predicciones = model.predict_proba(res)

resWithPred = res.copy()
resWithPred.loc[:,'Prob_Bot'] = pd.DataFrame(predicciones)[1].values
display(resWithPred.head(2))
# Joinear con nombres de usuarios, limpiar indices y joinear por orden de aparicion

# predicciones
ids = res.user_id.apply(lambda x: int(x)).values
ids = pd.DataFrame(ids)
probsBot = pd.DataFrame(predicciones)[[1]]
usuarios_prob = pd.concat([probsBot, ids], axis=1)
usuarios_prob.columns = ['Prob_Bot','user_id']
# usuarios_prob.loc[:,'Usuario'] = users.loc[usuarios_prob.user_id].screen_name

usuarios_prob.loc[:,'Usuario'] = usuarios_prob.user_id.apply(lambda x: users.loc[x].screen_name)

# Quedarnos con los que dan probabilidad mayor al 99% de ser bots segun el modelo basico
usuarios_prob.sort_values(by='Prob_Bot',ascending=False).head(5)

,user_id,tweetsCount,semanas,lun,mar,mie,jue,vie,sab,dom,...,num_mentions_median,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq,Prob_Bot
user_id,,,,,,,,,,,,,,,,,,,,,
102407528,102407528,170,10,3.166667,3.666667,3.833333,5.142857,4.625,3.666667,2.75,...,0.0,2776.739162008521,0.0,0.0,21574.074074074073,10000.0,31330.67640797719,0.0,30000.0,0.448755
1032436344759504896,1032436344759504896,87,4,3.666667,3.5,3.0,4.5,6.0,5.666667,10.333333,...,0,0,0,0,None,None,None,None,None,0.970073


,Prob_Bot,user_id,Usuario
1457,0.998038,948566185,miguelpasquau
1395,0.998005,937782543637762048,InfoRMAcf
1255,0.997391,731097872012353536,frenteobreroesp
724,0.997302,2870937975,usuariocastilla
651,0.997091,259935555,iFutboler0


In [131]:
yval = []
xval = np.linspace(0,1, 300)
for i in xval:
    cantUsuarios = len(usuarios_prob[usuarios_prob['Prob_Bot']>i])
    yval.append(cantUsuarios)
trace1 = go.Scatter(
    name='Inscriptos',
    x=xval,
    y=yval,
#     line={
#         'shape': 'spline',
#         'smoothing': 0.7,
#     }
)
layout = go.Layout(
    title=go.layout.Title(
        text='Cantidad bots',
        xref='paper',
        x=0
    ),
)
fig = go.Figure(data=[trace1], layout=layout)
print(len(usuarios_prob))
iplot(fig)

1486


In [200]:
usuarios_prob[usuarios_prob['Prob_Bot']>.01].sort_values(by='Prob_Bot', ascending=False).head(50)

,Prob_Bot,user_id,Usuario
1457,0.998038,948566185,miguelpasquau
1395,0.998005,937782543637762048,InfoRMAcf
1255,0.997391,731097872012353536,frenteobreroesp
724,0.997302,2870937975,usuariocastilla
651,0.997091,259935555,iFutboler0
702,0.996735,2825293828,lomaspopucom
734,0.996680,2874688017,Tu67Line
308,0.996676,151198642,revisbarcelona
1102,0.996475,584209647,RiverEPMA
1339,0.996378,842530170284244992,Lapreviaok_


In [134]:
def transform_x(df):
    df.loc[:, 'crawled_at'] = pd.DatetimeIndex(pd.to_datetime(df.crawled_at, utc=None))
    df.loc[:, 'created_at'] = pd.DatetimeIndex(pd.to_datetime(df.created_at, utc=None))
    
    
    account_life = df.crawled_at - df.created_at
    account_life_meses = np.round(account_life.dt.days / 30)
    account_life_meses = account_life_meses.replace(0, 1)
    account_life_semanas = np.round(account_life.dt.days / 7)
    account_life_semanas = account_life_semanas.replace(0, 1)
    numValues = ['tweet_count', 'listed_count', 'favs_count', 'followers_count', 'following_count']
    for i in numValues:
        colName_mes = i+'_mes'
        df.loc[:,colName_mes] = df[i].astype(float) / account_life_meses

        colName_semana = i+'_semana'
        df.loc[:,colName_semana] = df[i].astype(float) / account_life_semanas

    df.loc[:,'verified'] = ~df.verified.isna()
    df.loc[:,'description'] = ~df.description.isna()
    df.loc[:,'location'] = ~df.location.isna()
    df.loc[:,'url'] = ~df.url.isna()

    columnas = np.concatenate(
        [[x, x+'_mes', x+'_semana'] for x in numValues]
    )
    columnas = np.concatenate([
        [
            'verified',
            'description',
            'location',
            'url',
        ],
        columnas
    ])
    df = df[columnas]
    df = df.fillna(0)
    return df

In [144]:
with open('../modelo/modelPipeline.pkl', 'rb') as m_pred2:
    modelUsers = pickle.load(m_pred2)
modelUsers

Pipeline(memory=None,
     steps=[('cat', <catboost.core.CatBoostClassifier object at 0x000002013D53A128>)])

In [146]:
prediccionesUsers = modelUsers.predict_proba(transform_x(users))

In [157]:
usuarios_prob2 = pd.DataFrame(users.screen_name.to_list()).join(pd.DataFrame(prediccionesUsers)[1])
usuarios_prob2.columns = ['Usuario','Prob_Bot']
usuarios_prob2 = usuarios_prob2.set_index('Usuario', drop=False)

In [234]:
usuarios_prob.loc[:,'prob2'] = usuarios_prob.Usuario.apply(lambda x: usuarios_prob2.loc[x].Prob_Bot)
usuarios_prob.head()

,Prob_Bot,user_id,Usuario,prob2
0,0.448755,102407528,ebrenman,0.001329
1,0.970073,1032436344759504896,OlgaCutro,0.000643
2,0.592997,1032441989915140096,Agustinrgrau,0.002544
3,0.462507,1032737865602674690,etherealpilots,0.005604
4,0.952067,1032811143553789952,JoseMCr18192749,0.002917


In [166]:
trace1 = go.Scatter(
    name='Inscriptos mañana',
#     x=turnosDf.date.dt.hour,
    x = usuarios_prob.Prob_Bot,
    y = usuarios_prob.prob2,
    mode='markers', marker = dict(size = 10,opacity=0.7)
    
)
layout = go.Layout(
    title=go.layout.Title(
        text='', xref='paper', x=0
    ),
    xaxis = go.layout.XAxis(
        title=go.layout.xaxis.Title( text='Modelo complejo' )
    ),
    yaxis = go.layout.YAxis(
        title=go.layout.yaxis.Title( text='Modelo básico' )
    )
)
fig = go.Figure(data=[trace1], layout=layout)
iplot(fig)
# pio.write_image(fig, 'COMDOC - inscriptos por horario.png', width=1200, height=650)

In [246]:
usuarios_prob_id = usuarios_prob.copy()
usuarios_prob_id = usuarios_prob_id.set_index('user_id')
resWithPred2 = resWithPred.copy()
resWithPred2.loc[:,'prob2'] = resWithPred2.index.to_series().apply(lambda x: usuarios_prob_id.loc[int(x)].prob2)
resWithPred2.head()

,user_id,tweetsCount,semanas,lun,mar,mie,jue,vie,sab,dom,...,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq,Prob_Bot,prob2
user_id,,,,,,,,,,,,,,,,,,,,,
102407528,102407528,170,10,3.166667,3.666667,3.833333,5.142857,4.625,3.666667,2.75,...,2776.739162008521,0.0,0.0,21574.074074074073,10000.0,31330.67640797719,0.0,30000.0,0.448755,0.001329
1032436344759504896,1032436344759504896,87,4,3.666667,3.5,3.0,4.5,6.0,5.666667,10.333333,...,0,0,0,None,None,None,None,None,0.970073,0.000643
1032441989915140096,1032441989915140096,178,18,2.333333,2.166667,2.0,2.555556,1.666667,2.416667,1.636364,...,2141.5250999964933,0.0,0.0,2000.0,0.0,4634.275570907933,0.0,0.0,0.592997,0.002544
1032737865602674690,1032737865602674690,177,9,2.75,7.333333,2.0,7.333333,14.333333,11.75,6.5,...,0.0,0.0,0.0,1904.7619047619046,0.0,4822.86981579155,0.0,0.0,0.462507,0.005604
1032811143553789952,1032811143553789952,111,3,6.5,24.0,24.0,16.0,4.0,4.0,9.0,...,nan,0.0,0.0,0.0,0.0,nan,0.0,0.0,0.952067,0.002917


In [258]:
def getmarker(usr, sizeVar = False):
    col = False
    if(sizeVar != False):
        col = usr[sizeVar]
        col -= col.min()  # equivalent to df = df - df.min()
        col /= col.quantile(0.3)  # equivalent to df = df / df.max()
        col += [4 for x in range(len(col))]
    return dict(
        size = (lambda: 4 if sizeVar == False else usr[sizeVar])(),
        opacity=0.3
    )

def plotearVars(variables, sizeVar=False):
    colors = {'Humans':'red', 'SocialSpambot1':'blue', 'SocialSpambot2':'green', 'SocialSpambot3':'black'}
    resH = resWithPred2[resWithPred2['prob2'] < 0.01]
    res1 = resWithPred2[resWithPred2['prob2'] >= 0.01]
#     res2 = res[res.Cat=='SocialSpambot2']
#     res3 = res[res.Cat=='SocialSpambot3']
    
    trace1 = go.Scatter3d(
        name='Humans',
        x=resH[variables[0][0]],
        y=resH[variables[1][0]],
        z=resH[variables[2][0]],
        mode='markers',
        marker=getmarker(resH, sizeVar)
    )
    trace2 = go.Scatter3d(
        name='Bot',
        x=res1[variables[0][0]],
        y=res1[variables[1][0]],
        z=res1[variables[2][0]],
        mode='markers',
        marker=getmarker(res1, sizeVar)
    )
#     trace3 = go.Scatter3d(
#         name='SocialSpambot2',
#         x=res2[variables[0][0]],
#         y=res2[variables[1][0]],
#         z=res2[variables[2][0]],
#         mode='markers',
#         marker=getmarker(res2, sizeVar)
#     )
#     trace4 = go.Scatter3d(
#         name='SocialSpambot3',
#         x=res3[variables[0][0]],
#         y=res3[variables[1][0]],
#         z=res3[variables[2][0]],
#         mode='markers',
#         marker=getmarker(res3, sizeVar)
#     )
    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=30
        ),
        scene = dict(
            aspectmode='cube',
            xaxis = dict(
                range = (lambda: None if len(variables[0])==1 else variables[0][1])(),
                title=variables[0][0],
            ),
            yaxis = dict(
                range = (lambda: None if len(variables[1])==1 else variables[1][1])(),
                title=variables[1][0],
            ),
            zaxis = dict(
                range = (lambda: None if len(variables[2])==1 else variables[2][1])(),
                title= variables[2][0],
            ),
        )
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    iplot(fig)
    return trace1, fig, layout

In [259]:
trace, fig, layout = plotearVars([
    ['times_tq'],
    ['timesUser_tq', [0, 100000]],
    ['num_mentions_std', [0, 15000]],
])